In [1]:
import pandas as pd
import numpy as np

In [2]:
from dgutils.pandas import add_column

/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/pkg_resources/py2_warn.py:19: UserWarning: ************************************************************
You are running Setuptools on Python 2, which is no longer
supported and
>>> SETUPTOOLS WILL STOP WORKING <<<
in a subsequent release. Please ensure you are installing
Setuptools using pip 9.x or later or pin to `setuptools<45`
in your environment.
If you have done those things and are still encountering
this message, please comment in
https://github.com/pypa/setuptools/issues/1458
about the steps that led to this unsupported combination.
************************************************************
  sys.version_info < (3,) and warnings.warn("*" * 60 + msg + "*" * 60)
/Users/alicegao/anaconda2/envs/dg_work_py2/lib/python2.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

In [3]:
# load one of the raw data
# note that if loading ExE, "Check D-cell performance on interaction" won't work since we'll have no data points
# file_name = 'SGA_ExN_NxE.txt'
file_name = 'SGA_NxN.txt'

df = pd.read_csv('/Users/alicegao/work/psi-lab-sandbox/meetings/2020_02_11/d_cell/data/for_comparison/{}'.format(file_name), sep='\t')
df = df.rename(columns={
        'Query Strain ID': 's1',
        'Array Strain ID': 's2',
        'Query single mutant fitness (SMF)': 'f1',
        'Array SMF': 'f2',
        'Genetic interaction score (ε)': 'interaction',
    'P-value': 'pval',
    'Double mutant fitness': 'fd',
    'Double mutant fitness standard deviation': 'fstd',
    })

## Investigate relationship between single KO fitness and double KO fitness

In [4]:
df = add_column(df, 'interaction_calculated', ['f1', 'f2', 'fd'],
               lambda f1, f2, fd: fd - f1 * f2)

In [5]:
# no data point with significant double fitness has a huge difference 
# between the fitness from the raw dataset and the one computed using above formula
assert len(df[((df['interaction'] - df['interaction_calculated']) > 1e-3) & (df['pval'] < 0.05)]) == 0

## Check D-cell performance on interaction

In [6]:
# growth prediction
df_dcell_pred = pd.read_csv('/Users/alicegao/work/psi-lab-sandbox/meetings/2020_02_11/d_cell/data/for_comparison/costonzo_2009_pvalue005_prediction_growth',
                           sep='\t', header=None, names=['g1', 'g2', 'pred'])

In [7]:
# de-dup raw data for join

def get_gene_id(strain_id):
    # extra gene ID from strain ID
    gene_id, _ = strain_id.split('_')
    return gene_id

df = add_column(df, 'g1', ['s1'], get_gene_id)
df = add_column(df, 'g2', ['s2'], get_gene_id)
# take median of examples with the same gene pair, so that we don't have duplicates
df = df[['g1', 'g2', 'interaction', 'fd']].groupby(by=['g1', 'g2'], as_index=False).agg('median')

In [8]:
df_dcell_pred = pd.merge(df_dcell_pred, df)  # inner join

In [9]:
df_dcell_pred['pred'].corr(df_dcell_pred['interaction'])

0.026616867348584622

In [10]:
df_dcell_pred['pred'].corr(df_dcell_pred['fd'])

0.2311667247791089

In [11]:
# gi prediction
df_dcell_pred = pd.read_csv('/Users/alicegao/data/tmp/costonzo_2009_pvalue005_prediction',
                           sep='\t', header=None, names=['g1', 'g2', 'pred'])

In [12]:
df_dcell_pred = pd.merge(df_dcell_pred, df)  # inner join

In [13]:
df_dcell_pred['pred'].corr(df_dcell_pred['interaction'])

0.4892228685092307

In [14]:
df_dcell_pred['pred'].corr(df_dcell_pred['fd'])

0.21512434519724072

## Check correlation between fitness and interaction in raw data

In [15]:
df['interaction'].corr(df['fd'])

0.2743129604320869